# Projet Finla Maxime Bouvry, Le VendéeGlobe

In [335]:
from shutil import copyfileobj
from urllib import request
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
import html5lib
import regex as re
import os
import numpy as np
import time
import datetime
from unidecode import unidecode
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import time

## 1 -- Acquisition et chargement des données : 3 points

1.1 Données relatives à la course

In [336]:
var = pd.read_html("https://www.vendeeglobe.org/fr/classement")
[df.shape for df in var]
df = var[0]
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0  \
  Unnamed: 0_level_1 Unnamed: 1_level_1   
0                NaN                NaN   
1                  1                NaN   
2                  2                NaN   
3                  3                NaN   
4                  4                NaN   

                                  Unnamed: 2_level_0  \
                                  Unnamed: 2_level_1   
0                                                NaN   
1  Au regard du temps consacré au sauvetage de Ke...   
2                                                NaN   
3                                                NaN   
4  Au regard du temps consacré au sauvetage de Ke...   

                  Skipper/Bateau  \
                  Skipper/Bateau   
0                            NaN   
1  Yannick BESTAVENMaître CoQ IV   
2            Charlie DALINAPIVIA   
3    Louis BURTONBUREAU VALLEE 2   
4         Jean LE CAMYes We Cam!   

  Foils ? 19 IMOCA sont équipés de plans porteurs incurvés  \
  Foils ? 19 IMOCA sont équipés de plans porteurs incurvés   
0                                                NaN         
1                                                Oui         
2                                                Oui         
3                                                Oui         
4                                                Non         

  Vitesse ? Un nœud est égal à un mille marin par heure, soit 1,852 km/h  \
  Vitesse ? Un nœud est égal à un mille marin par heure, soit 1,852 km/h   
0                                                NaN                       
1                 Date arrivée : 28/01/2021 03:19:46                       
2                 Date arrivée : 27/01/2021 19:35:47                       
3                 Date arrivée : 27/01/2021 23:45:12                       
4                 Date arrivée : 28/01/2021 19:19:55                       

  VMC  ? La VMC (Velocity Made Course) est la vitesse de rapprochement au but  \
  VMC  ? La VMC (Velocity Made Course) est la vitesse de rapprochement au but   
0                                                NaN                            
1                 Date arrivée : 28/01/2021 03:19:46                            
2                 Date arrivée : 27/01/2021 19:35:47                            
3                 Date arrivée : 27/01/2021 23:45:12                            
4                 Date arrivée : 28/01/2021 19:19:55                            

                              Cap                        Lat/Long  \
                              Cap                        Lat/Long   
0                             NaN                             NaN   
1  Temps course : 80j 03h 44m 46s  Temps course : 80j 03h 44m 46s   
2  Temps course : 80j 06h 15m 47s  Temps course : 80j 06h 15m 47s   
3  Temps course : 80j 10h 25m 12s  Temps course : 80j 10h 25m 12s   
4  Temps course : 80j 13h 44m 55s  Temps course : 80j 13h 44m 55s   

                         Distance                                  \
                      à l'arrivée             Unnamed: 10_level_1   
0                             NaN                             NaN   
1  Temps course : 80j 03h 44m 46s  Temps course : 80j 03h 44m 46s   
2  Temps course : 80j 06h 15m 47s  Temps course : 80j 06h 15m 47s   
3  Temps course : 80j 10h 25m 12s  Temps course : 80j 10h 25m 12s   
4  Temps course : 80j 13h 44m 55s  Temps course : 80j 13h 44m 55s   

                                                                   
                        Au leader                  Au poursuivant  
0                             NaN                             NaN  
1  Temps course : 80j 03h 44m 46s  Temps course : 80j 03h 44m 46s  
2  Temps course : 80j 06h 15m 47s  Temps course : 80j 06h 15m 47s  
3  Temps course : 80j 10h 25m 12s  Temps course : 80j 10h 25m 12s  
4  Temps course : 80j 13h 44m 55s  Temps course : 80j 13h 44m 55s

In [337]:
# Relevé des distances à 4 8 11 14 21h du 08/11/2020 au 05/03/2021
# récuper la liste des dates --> Création de l'url puis télechargement. 
# https://www.vendeeglobe.org/fr/classement/20210305_080000
# https://www.vendeeglobe.org/fr/classement/20210305_040000
# https://www.vendeeglobe.org/fr/classement/20210304_210000


In [338]:
#le premier arrive le 27 janv a 17h 
url = "https://www.vendeeglobe.org/fr/classement"
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
L=[]
df=pd.DataFrame()
urls=soup.find_all('option')
values = [o["value"] for o in urls if o["value"]]
for value in values:
    L.append(f'https://www.vendeeglobe.org/fr/classement/{value}')
# matches = [match for match in L if "https://www.vendeeglobe.org/fr/classement/20210127_170000" in match]
L.index('https://www.vendeeglobe.org/fr/classement/20210127_170000')
L1=L[219:]
len(L1)
L1[1]=='https://www.vendeeglobe.org/fr/classement/20210127_140000'

True

In [339]:
L2=L1[1]
df1=pd.DataFrame()
for i,t in enumerate(L1) : 
    var0 = pd.read_html(t)
  # var1 = pd.read_html(L[1])
    df_tmp0 = var0[0]
  # df_tmp1 = var1=[0]
    df1=pd.concat([df1,df_tmp0], ignore_index=True)
   #pd.concat([df,df_tmp1], ignore_index=True)

df1=df1.droplevel(0,axis=1)
df1=df1.rename(columns={"Unnamed: 0_level_1": "R2", 
                      "Unnamed: 1_level_1": "R1",
                      "Unnamed: 2_level_1":"R3",
                      "Unnamed: 10_level_1" : "distance parcourue",
                      "Foils ? 19 IMOCA sont équipés de plans porteurs incurvés" : "Foil",
                      "Vitesse ? Un nœud est égal à un mille marin par heure, soit 1,852 km/h" : 'vitesse',
                      "VMC  ? La VMC (Velocity Made Course) est la vitesse de rapprochement au but" : 'VMC',
                      })
df1=df1.drop('R1',axis=1)
df1=df1.drop('R2',axis=1)
df1=df1.drop('R3',axis=1)
df1=df1.drop(0,axis=0)
df1


,Skipper/Bateau,Foil,vitesse,VMC,Cap,Lat/Long,à l'arrivée,distance parcourue,Au leader,Au poursuivant
1,Charlie DALINAPIVIA,Oui,Date arrivée : 27/01/2021 19:35:47,Date arrivée : 27/01/2021 19:35:47,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s
2,Louis BURTONBUREAU VALLEE 2,Oui,18.14 nds 33.6 km/h,16.21 nds,114°,46° 3'1'' N 4° 0'13'' O,95.68 nm 177.2 km,99.6%,0 nm 0 km,53.69 nm 99.43 km
3,Boris HERRMANNSEAEXPLORER - YACHT CLUB DE MONACO,Oui,13.59 nds 25.17 km/h,13.55 nds,55°,44° 54'54'' N 4° 33'38'' O,149.36 nm 276.61 km,99.4%,53.69 nm 99.43 km,33.89 nm 62.76 km
4,Thomas RUYANTLinkedOut,Oui,17.57 nds 32.54 km/h,17.51 nds,106°,47° 9'57'' N 6° 7'25'' O,183.26 nm 339.4 km,99.3%,87.58 nm 162.2 km,22.9 nm 42.41 km
5,Yannick BESTAVENMaître CoQ IV,Oui,19.38 nds 35.89 km/h,19.38 nds,104°,47° 29'27'' N 6° 35'10'' O,206.15 nm 381.79 km,99.2%,110.48 nm 204.61 km,98.25 nm 181.96 km
...,...,...,...,...,...,...,...,...,...,...
16553,Ari HUUSELASTARK,Non,0 nds 0 km/h,0 nds,0°,46° 30'8'' N 1° 47'24'' O,24296.6 nm 44997.3 km,0%,0 nm 0 km,NaN
16554,Stéphane LE DIRAISONTIME FOR OCEANS,Oui,0 nds 0 km/h,0 nds,0°,46° 30'8'' N 1° 47'24'' O,24296.6 nm 44997.3 km,0%,0 nm 0 km,NaN
16555,Alexia BARRIERTSE - 4MYPLANET,Non,0 nds 0 km/h,0 nds,0°,46° 30'8'' N 1° 47'24'' O,24296.6 nm 44997.3 km,0%,0 nm 0 km,NaN
16556,Maxime SORELV and B-MAYENNE,Non,0 nds 0 km/h,0 nds,0°,46° 30'8'' N 1° 47'24'' O,24296.6 nm 44997.3 km,0%,0 nm 0 km,NaN


1.2 Données relatives aux caractéristiques du bateau

In [340]:
url = "https://www.vendeeglobe.org/fr/glossaire"
res = requests.get(url)
print(url)
soup = BeautifulSoup(res.content, 'html.parser')
df3=pd.DataFrame()
nom_equipe=[]
cara = soup.find_all(class_="boats-list__popup-infos")
for i,t in enumerate(cara): # i indice de l'elem et t l'elem 
    nom_equipe = t.find("h3").text
    C1=t.find(class_="boats-list__popup-specs-list").text
    C2 = t.find_all("li")
    C2=[elem.text for elem in C2] #for elem in C2 : C2.append[elem.text] déclarer au préalable C2=[]
    C={}
    for elem in C2 : 
       C3=elem.split(' : ',1)
       C4=dict([C3])
       C.update(C4)
    if not 'Anciens noms du bateau' in C : 
         C['Anciens noms du bateau']=''
    df3_tmp=pd.DataFrame(C,index=[0])
    #df3=df3+df3_tmp
    df3=pd.concat([df3,df3_tmp])    
    
df3.head()
df3=df3.reset_index()
df3=df3.drop('index', axis=1)
df3.head()


https://www.vendeeglobe.org/fr/glossaire


,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,08,,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2


In [341]:
cara1 = soup.find_all(class_="boats-list__skipper-name")
C5=[elem.text for elem in cara1]
C5
df4=pd.DataFrame(C5)
df4
# for i,t in enumerate(C2) : 
#     df3['Nom du skipper']=C2[i]
# df3
df3=pd.concat([df3,df4],axis=1)
df3.head()


,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant,0
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2,Fabrice AMEDEO
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2,Romain ATTANASIO
2,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2,Alexia BARRIER
3,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2,Yannick BESTAVEN
4,08,,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2,Jérémie BEYOU


In [342]:
df3=df3.rename(columns={0 : 'Skipper'})
first_column = df3.pop('Skipper')
df3.insert(0, 'Skipper', first_column)
df3


,Skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,Fabrice AMEDEO,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,Romain ATTANASIO,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,Alexia BARRIER,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,Yannick BESTAVEN,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,Jérémie BEYOU,08,,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,Arnaud BOISSIÈRES,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,Louis BURTON,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,Didac COSTA,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,Manuel COUSIN,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,Clarisse CREMER,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2


## Préparation des données : 5 points

1.2 Données relatives aux caractéristiques du bateau

In [343]:
df = df3.copy()
df = df.rename(columns={
    'Date de lancement': 'Date de lancement',
    'Longueur': 'Longueur',
    'Largeur': 'Largeur',
    'Déplacement (poids)': 'poids',
    'Nombre de dérives': 'dérives',  
    'Hauteur mât': 'mat',  
    'Voile quille': 'quille',  
    'Surface de voiles au près': 'voiles au près',  
    'Surface de voiles au portant': 'voiles au portant',    
})
df

,Skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,poids,dérives,mat,quille,voiles au près,voiles au portant
0,Fabrice AMEDEO,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,Romain ATTANASIO,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,Alexia BARRIER,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,Yannick BESTAVEN,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,Jérémie BEYOU,08,,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,Arnaud BOISSIÈRES,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,Louis BURTON,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,Didac COSTA,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,Manuel COUSIN,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,Clarisse CREMER,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2


In [344]:
df['Longueur'] = df['Longueur'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['Largeur'] = df['Largeur'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['poids'] = df['poids'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)
df['mat'] = df['mat'].apply(lambda x: float(re.sub(r"([^0-9,]+)",'', x).replace(',', '.')) if (isinstance(x, str) and any([l.isdigit() for l in x])) else np.nan)

df['voiles au près'] = df['voiles au près'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)
df['voiles au portant'] = df['voiles au portant'].apply(lambda x: float(x[:3]) if (isinstance(x, str) and len(x)>3) else np.nan)

keel2no = {keel:no for no, keel in enumerate(df['quille'].dropna().unique().tolist())}
no2keels = {v:k for k,v in keel2no.items()}
df['quille'] = df['quille'].apply(lambda x: keel2no[x] if isinstance(x, str) else np.nan)
df = df.replace('', np.nan)
df


,Skipper,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,poids,dérives,mat,quille,voiles au près,voiles au portant
0,Fabrice AMEDEO,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,18.28,5.85,"4,50 m",7.0,foils,29.0,0.0,320.0,570.0
1,Romain ATTANASIO,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,18.28,5.80,"4,50m",9.0,2,28.0,1.0,280.0,560.0
2,Alexia BARRIER,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,18.28,5.54,"4,50m",9.0,2,29.0,2.0,260.0,580.0
3,Yannick BESTAVEN,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,18.28,5.80,"4,50 m",8.0,foils,29.0,3.0,310.0,550.0
4,Jérémie BEYOU,08,NaN,VPLP,CDK Technologies,18 Août 2018,18.28,5.85,"4,50 m",8.0,foils,29.0,2.0,320.0,600.0
5,Arnaud BOISSIÈRES,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,18.28,5.65,"4,50 m",7.9,foils,29.0,4.0,300.0,610.0
6,Louis BURTON,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,18.28,5.80,"4,50 m",7.6,foils,28.0,2.0,300.0,600.0
7,Didac COSTA,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,18.28,5.30,"4,50 m",8.9,2,26.0,2.0,240.0,470.0
8,Manuel COUSIN,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,18.28,5.80,"4,50 m",9.0,2 asymétriques,28.5,5.0,270.0,560.0
9,Clarisse CREMER,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,18.28,5.70,"4,5 m",7.7,2,29.0,1.0,340.0,570.0


In [345]:
# df10=df['Date de lancement']
# df10

In [346]:
df5=df.copy()
df6=df5[['Skipper', 'Longueur', 'Largeur', 'poids', 'dérives', 'voiles au près', 'voiles au portant','mat','quille']]
df7=df5[['Skipper', 'Numéro de voile', 'Anciens noms du bateau', 'Architecte', 'Chantier']]

In [347]:
df6['dérives']=df6['dérives'].apply(lambda x : 1 if x=='foils' or x=='foiler' else 0)
df6['poids']=df6['poids'].apply(lambda x : round(df6['poids'].mean(),2) if np.nan else x)
df6=df6.drop(index=16)
df6=df6.reset_index()
df6=df6.drop(columns='index')


C:\Users\maxim\AppData\Local\Temp\ipykernel_19976\941166364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['dérives']=df6['dérives'].apply(lambda x : 1 if x=='foils' or x=='foiler' else 0)
C:\Users\maxim\AppData\Local\Temp\ipykernel_19976\941166364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['poids']=df6['poids'].apply(lambda x : round(df6['poids'].mean(),2) if np.nan else x)


In [348]:
df6

,Skipper,Longueur,Largeur,poids,dérives,voiles au près,voiles au portant,mat,quille
0,Fabrice AMEDEO,18.28,5.85,8.14,1,320.0,570.0,29.0,0.0
1,Romain ATTANASIO,18.28,5.80,8.14,0,280.0,560.0,28.0,1.0
2,Alexia BARRIER,18.28,5.54,8.14,0,260.0,580.0,29.0,2.0
3,Yannick BESTAVEN,18.28,5.80,8.14,1,310.0,550.0,29.0,3.0
4,Jérémie BEYOU,18.28,5.85,8.14,1,320.0,600.0,29.0,2.0
5,Arnaud BOISSIÈRES,18.28,5.65,8.14,1,300.0,610.0,29.0,4.0
6,Louis BURTON,18.28,5.80,8.14,1,300.0,600.0,28.0,2.0
7,Didac COSTA,18.28,5.30,8.14,0,240.0,470.0,26.0,2.0
8,Manuel COUSIN,18.28,5.80,8.14,0,270.0,560.0,28.5,5.0
9,Clarisse CREMER,18.28,5.70,8.14,0,340.0,570.0,29.0,1.0


2.2 Données relatives à la course


In [349]:

def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'E' or direction == 'S':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    parts = re.split('[^\d\w]+', dms)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[3])
 
    return (lat)

dd = parse_dms("46° 3'1'' N 4° 0'13'' O")



In [352]:
L2=L1[1]
df11=df1.head(32)
# # tri selon l'altitude inverse
# df11.sort_values("Altitude Moyenne", ascending=False).head()
# df11=df11.drop(columns='Skipper/Bateau')
# df11=pd.concat([df11,df8],axis=1)
df11

,Skipper/Bateau,Foil,vitesse,VMC,Cap,Lat/Long,à l'arrivée,distance parcourue,Au leader,Au poursuivant
1,Charlie DALINAPIVIA,Oui,Date arrivée : 27/01/2021 19:35:47,Date arrivée : 27/01/2021 19:35:47,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s,Temps course : 80j 06h 15m 47s
2,Louis BURTONBUREAU VALLEE 2,Oui,18.14 nds 33.6 km/h,16.21 nds,114°,46° 3'1'' N 4° 0'13'' O,95.68 nm 177.2 km,99.6%,0 nm 0 km,53.69 nm 99.43 km
3,Boris HERRMANNSEAEXPLORER - YACHT CLUB DE MONACO,Oui,13.59 nds 25.17 km/h,13.55 nds,55°,44° 54'54'' N 4° 33'38'' O,149.36 nm 276.61 km,99.4%,53.69 nm 99.43 km,33.89 nm 62.76 km
4,Thomas RUYANTLinkedOut,Oui,17.57 nds 32.54 km/h,17.51 nds,106°,47° 9'57'' N 6° 7'25'' O,183.26 nm 339.4 km,99.3%,87.58 nm 162.2 km,22.9 nm 42.41 km
5,Yannick BESTAVENMaître CoQ IV,Oui,19.38 nds 35.89 km/h,19.38 nds,104°,47° 29'27'' N 6° 35'10'' O,206.15 nm 381.79 km,99.2%,110.48 nm 204.61 km,98.25 nm 181.96 km
6,Damien SEGUINGROUPE APICIL,Non,18.07 nds 33.47 km/h,17.99 nds,75°,46° 4'12'' N 9° 5'39'' O,304.41 nm 563.77 km,98.8%,208.73 nm 386.57 km,11.61 nm 21.5 km
7,Giancarlo PEDOTEPRYSMIAN GROUP,Oui,20.3 nds 37.6 km/h,20.27 nds,84°,46° 41'18'' N 9° 26'2'' O,316.01 nm 585.25 km,98.7%,220.34 nm 408.07 km,144.22 nm 267.1 km
8,Jean LE CAMYes We Cam!,Non,17.18 nds 31.82 km/h,17.15 nds,70°,45° 0'4'' N 12° 34'6'' O,460.23 nm 852.35 km,98.1%,364.56 nm 675.17 km,169.15 nm 313.27 km
9,Benjamin DUTREUXOMIA - WATER FAMILY,Non,18.51 nds 34.28 km/h,18.35 nds,62°,43° 57'22'' N 16° 15'0'' O,629.38 nm 1165.61 km,97.4%,533.71 nm 988.43 km,219.1 nm 405.77 km
10,Maxime SORELV and B-MAYENNE,Non,16.46 nds 30.48 km/h,16.46 nds,59°,40° 13'43'' N 19° 16'48'' O,848.49 nm 1571.4 km,96.5%,752.81 nm 1394.2 km,134.48 nm 249.06 km


## Analyses et story telling : 7 points

Le point majeur de ce Vendée Globe 2020-2021 est l'arrivée de bateau à foil dans la compétition. Il faut prendre aussi en compte un point important qui est l'autorisation des réparations par le jury. En effet, le classeement du temps d'arrivée et différent du classement final qui prend en compte ce paramètre. 
La casse des bateaux n'est pas pris en compte dans les données ci-dessus ce qui est une variable explicative majeur dans la navigation pour chaque bateau. 

Ci dessous le classement final de la course, basons-nous sur celui la, pour la suite des explications. On remarque que le premier arrivée fini deuxième de la course car Yannick s'est vu attribué 10h15 par je jury

In [353]:
var = pd.read_html("https://fr.wikipedia.org/wiki/Vend%C3%A9e_Globe_2020-2021")
[df.shape for df in var]
df9=var[7]
df9

,#,Nom,Nom du bateau,Arrivé le,Tempsaccordépar le jury,Temps,Écart
0,1,Yannick Bestaven,Maître CoQ IV,28 janvier 04 h 19,10 h 15 min,080 j 03 h 44 min 46 s,NaN
1,2,Charlie Dalin,Apivia,27 janvier 20 h 35,NaN,080 j 06 h 15 min 47 s,02 h 31 min
2,3,Louis Burton,Bureau Vallée 2,28 janvier 00 h 45,NaN,080 j 10 h 25 min 12 s,06 h 40 min
3,4,Jean Le Cam,Yes We Cam !,28 janvier 20 h 19,16 h 15 min,080 j 13 h 44 min 55 s,10 h 00 min
4,5,Boris Herrmann,Seaexplorer Yacht club de Monaco,28 janvier 11 h 19,06 h00 min,080 j 14 h 59 min 45 s,11 h 14 min
5,6,Thomas Ruyant,Linked Out,28 janvier 05 h 42,NaN,080 j 15 h 22 min 01 s,11 h 37 min
6,7,Damien Seguin,Groupe Apicil,28 janvier 12 h 18,NaN,080 j 21 h 58 min 20 s,18 h 13 min
7,8,Giancarlo Pedote,Prysmian Group,28 janvier 13 h 02,NaN,080 j 22 h 42 min 20 s,18 h 57 min
8,9,Benjamin Dutreux,Omia Water Family,29 janvier 10 h 05,NaN,081 j 20 h 45 min 20 s,1 j 17 h 01 min
9,10,Maxime Sorel,V and B Mayenne,30 janvier 04 h 50,NaN,082 j 14 h 30 min 15 s,2 j 10 h 45 min


In [400]:
d={'col1' : [26,14,24,1,13,15,3,20,23,12,2,27,28,9,29,21,19,5,25,30,4,18,22,8,17,6,7,16,31,10,32,11,33]}
df14=pd.DataFrame(d)
df13=pd.concat([df6,df14],axis=1)

In [401]:
df13=df13.reset_index()
df13=df13.sort_values('col1')
df13

,index,Skipper,Longueur,Largeur,poids,dérives,voiles au près,voiles au portant,mat,quille,col1
3,3,Yannick BESTAVEN,18.28,5.80,8.14,1,310.0,550.0,29.0,3.0,1
10,10,Charlie DALIN,18.28,5.85,8.14,1,350.0,560.0,29.0,2.0,2
6,6,Louis BURTON,18.28,5.80,8.14,1,300.0,600.0,28.0,2.0,3
20,20,Jean LE CAM,18.28,5.90,8.14,0,300.0,620.0,28.0,1.0,4
17,17,Boris HERRMANN,18.28,5.70,8.14,1,290.0,490.0,29.0,2.0,5
25,25,Thomas RUYANT,18.28,5.85,8.14,1,350.0,560.0,29.0,1.0,6
26,26,Damien SEGUIN,18.28,5.85,8.14,0,350.0,610.0,29.0,1.0,7
23,23,Giancarlo PEDOTE,18.28,5.80,8.14,1,300.0,600.0,29.0,1.0,8
13,13,Benjamin DUTREUX,18.28,5.85,8.14,0,300.0,700.0,29.0,1.0,9
29,29,Maxime SOREL,18.28,5.50,8.14,0,365.0,700.0,29.0,2.0,10


In [402]:

# y= df.loc[:,160]
# X=df.loc[:,0:159]
y= df13['col1']
x=df13.copy()
x=x.drop(['index'],axis=1)
x=x.drop(['Skipper'],axis=1)
x=x.fillna(0)
# X=df13.loc[:,2,12]


In [403]:
x

,Longueur,Largeur,poids,dérives,voiles au près,voiles au portant,mat,quille,col1
3,18.28,5.80,8.14,1,310.0,550.0,29.0,3.0,1
10,18.28,5.85,8.14,1,350.0,560.0,29.0,2.0,2
6,18.28,5.80,8.14,1,300.0,600.0,28.0,2.0,3
20,18.28,5.90,8.14,0,300.0,620.0,28.0,1.0,4
17,18.28,5.70,8.14,1,290.0,490.0,29.0,2.0,5
25,18.28,5.85,8.14,1,350.0,560.0,29.0,1.0,6
26,18.28,5.85,8.14,0,350.0,610.0,29.0,1.0,7
23,18.28,5.80,8.14,1,300.0,600.0,29.0,1.0,8
13,18.28,5.85,8.14,0,300.0,700.0,29.0,1.0,9
29,18.28,5.50,8.14,0,365.0,700.0,29.0,2.0,10


In [404]:
regl=LinearRegression()
regl.fit(x,y)
print(regl.intercept_)
print(regl.coef_)

5.3290705182007514e-14
[ 0.00000000e+00  4.26325641e-14 -2.22044605e-16 -1.11482004e-14
  2.22044605e-16 -4.44089210e-16 -3.57179564e-15 -1.38951350e-15
  1.00000000e+00]


Les coeficients sont difficilements explicables au vu de la faible différence des valeurs entre les bateaux, les foils n'expliquent pas beaucoup plus le lien entre les données relatives au bateau et le classsement. Cela s'explique par la présence de foil sur les 5 derniers bateaux